.. _nb_mixed_variable:

## Mixed Variable Problem

In some cases, variables might have different types; some might be real, and others integer-valued. For those cases, different evolutionary operators must be applied to a different type of variables. In **pymoo** this is supported by instantiating operators with a mask. Let us consider the following optimization problem with two variables:

In [1]:
from pymoo.core.problem import ElementwiseProblem

class MixedVariableProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        vars = {
            "z": Real(bounds=(-5, 5)),
            "y": Integer(bounds=(-2, 2)),
        }
        super().__init__(vars=vars, n_obj=1, n_ieq_constr=1, **kwargs)

    def _evaluate(self, X, out, *args, **kwargs):
        z, y = X["z"], X["y"]
        out["F"] = z + y
        out["G"] = 3 - z - y

We would like to keep the first variable in integer and the second variable a real value; therefore, we first define a mask that defines the type for each variable:

Furthermore, all operators that are necessary to run the algorithm must be defined. The mixed variable classes allow the use of existing operator implementations and apply inside the mask during running time. The keys of the dictionary must match the keywords used in the mask. Internally, the mating considers each type of the mask separately, applies the operator, and finally concatenates the results. 

In [2]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.mixed import MixedVariableMating, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.core.variable import Real, Integer
from pymoo.optimize import minimize


problem = MixedVariableProblem()

dupl = MixedVariableDuplicateElimination()

algorithm = GA(sampling=MixedVariableSampling(),
               mating=MixedVariableMating(eliminate_duplicates=dupl),
               eliminate_duplicates=dupl)

res = minimize(problem,
               algorithm,
               ('n_gen', 50),
               seed=1,
               save_history=True,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = {'z': 2.000026156077974, 'y': 1}
F = [3.00002616]
